In [2]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, balanced_accuracy_score
import networkx as nx
from statannot import add_stat_annotation
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
kmf = KaplanMeierFitter()

In [3]:
sys.path

['/home/jijoo/theor/lib/python35.zip',
 '/home/jijoo/theor/lib/python3.5',
 '/home/jijoo/theor/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/jijoo/theor/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '',
 '/home/jijoo/theor/lib/python3.5/site-packages',
 '/home/jijoo/theor/lib/python3.5/site-packages/IPython/extensions',
 '/home/jijoo/.ipython',
 '/home/jijoo/tmp/pycharm_project_225']

In [4]:
oncoprint_pd = pd.read_csv('../data/BLCA_200919/CCLE_DATA_oncoprint_expression.tsv', sep='\t')
gene_node_pd = pd.read_csv('../data/BLCA_200919/Grieco_gene_node_oncoKB_mapping.csv')
table_original_network = pd.read_csv('../data/Grieco_table_for_original_network.tsv', sep='\t').sort_values(by='node')
# print(oncoprint_pd.head())

In [5]:
oncoprint_mut_pd = oncoprint_pd[oncoprint_pd['track_type']=='MUTATIONS']
oncoprint_cna_pd = oncoprint_pd[oncoprint_pd['track_type']=='CNA']
oncoprint_mrna_pd = oncoprint_pd[oncoprint_pd['track_type']=='MRNA']
oncoprint_mut_pd = oncoprint_pd[oncoprint_pd['track_type']=='MUTATIONS']
oncoprint_cna_pd = oncoprint_pd[oncoprint_pd['track_type']=='CNA']
oncoprint_mrna_pd = oncoprint_pd[oncoprint_pd['track_type']=='MRNA']
oncoprint_mrna_melt_pd = pd.melt(oncoprint_mrna_pd, id_vars=['track_name', 'track_type'], var_name='Case ID', value_name='event')
oncoprint_mrna_pivot_pd = pd.pivot(oncoprint_mrna_melt_pd[['track_name', 'Case ID', 'event']], index='Case ID', columns='track_name', values='event').sort_index()
oncoprint_cna_melt_pd = pd.melt(oncoprint_cna_pd, id_vars=['track_name', 'track_type'], var_name='Case ID', value_name='event')
oncoprint_cna_pivot_pd = pd.pivot(oncoprint_cna_melt_pd[['track_name', 'Case ID', 'event']], index='Case ID', columns='track_name', values='event').sort_index()
oncoprint_mut_melt_pd = pd.melt(oncoprint_mut_pd, id_vars=['track_name', 'track_type'], var_name='Case ID', value_name='event')
oncoprint_mut_pivot_pd = pd.pivot(oncoprint_mut_melt_pd[['track_name', 'Case ID', 'event']], index='Case ID', columns='track_name', values='event').sort_index()

In [6]:
# cna_mrna_consistent_pivot_pd = ((oncoprint_cna_pivot_pd == 'Amplification') & (oncoprint_mrna_pivot_pd  == 'mRNA High')) | ((oncoprint_cna_pivot_pd  == 'Deep Deletion'))
cna_mrna_consistent_pivot_pd = ((oncoprint_cna_pivot_pd == 'Amplification') & (oncoprint_mrna_pivot_pd  == 'mRNA High')) | ((oncoprint_cna_pivot_pd  == 'Deep Deletion') & (oncoprint_mrna_pivot_pd == 'mRNA Low'))
# print(cna_mrna_consistent_pivot_pd.head())
mrna_not_low_pivot_pd = ~(oncoprint_mrna_pivot_pd == 'mRNA Low')
# print(mrna_not_low_pivot_pd.head())
alteration_cna_melt_pd = pd.melt(oncoprint_cna_pivot_pd[cna_mrna_consistent_pivot_pd].reset_index(), id_vars='Case ID', var_name='track_name', value_name='event')
alteration_mut_melt_pd = pd.melt(oncoprint_mut_pivot_pd[mrna_not_low_pivot_pd].reset_index(), id_vars='Case ID', var_name='track_name', value_name='event')
alteration_cna_melt_pd['track_type'] = 'CNA'
alteration_mut_melt_pd['track_type'] = 'MUTATIONS'

alteration_driver_melt_pd = alteration_mut_melt_pd[alteration_mut_melt_pd['event'].str.contains('passenger')==False]
alteration_cna_driver_melt_pd = pd.concat([alteration_cna_melt_pd, alteration_driver_melt_pd])

# alteration_cna_mut_melt_pd = pd.concat([alteration_cna_melt_pd, alteration_mut_melt_pd])
oncoprint_node_pd = pd.merge(alteration_cna_driver_melt_pd, gene_node_pd, left_on='track_name', right_on='gene_symbol')
oncoprint_node_class_pd = pd.merge(oncoprint_node_pd.dropna(), table_original_network[['node', 'Class']], left_on='node_name', right_on='node')
oncoprint_node_class_pd['mutation_sign'] = 1*(oncoprint_node_class_pd['oncoKB'] == 'Oncogene') - 1*(oncoprint_node_class_pd['oncoKB'] == 'Tumor Suppressor')
# oncoprint_node_class_pd.head()

oncoprint_blca_pd = oncoprint_node_class_pd[oncoprint_node_class_pd['Case ID'].str.contains('URINAR')]
oncoprint_blca_pd['cell line'] = oncoprint_blca_pd['Case ID'].str.replace('_URINARY_TRACT', '')
print(oncoprint_blca_pd.head())

                  Case ID track_name                                  event  \
14   KU1919_URINARY_TRACT       AKT1    Missense Mutation (putative driver)   
165   JMSU1_URINARY_TRACT        ATM  Truncating mutation (putative driver)   
202   UMUC3_URINARY_TRACT        ATM  Truncating mutation (putative driver)   
221  HT1376_URINARY_TRACT     CDKN1A  Truncating mutation (putative driver)   
222   KMBC2_URINARY_TRACT     CDKN1A  Truncating mutation (putative driver)   

    track_type gene_symbol node_name            oncoKB node Class  \
14   MUTATIONS        AKT1       AKT          Oncogene  AKT    C1   
165  MUTATIONS         ATM       ATM  Tumor Suppressor  ATM    C0   
202  MUTATIONS         ATM       ATM  Tumor Suppressor  ATM    C0   
221  MUTATIONS      CDKN1A       p21  Tumor Suppressor  p21    C3   
222  MUTATIONS      CDKN1A       p21  Tumor Suppressor  p21    C3   

     mutation_sign cell line  
14               1    KU1919  
165             -1     JMSU1  
202             -

/home/jijoo/theor/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
# oncoprint_blca_node_pd['mutation_sign'] = 1*(oncoprint_blca_node_pd['oncoKB'] == 'Oncogene') - 1*(oncoprint_blca_node_pd['oncoKB'] == 'Tumor Suppressor')
# cna_blca_mutation_pd = oncoprint_blca_node_pd.loc[np.logical_or(oncoprint_blca_node_pd['track_type']=='CNA', oncoprint_blca_node_pd['track_type']=='MUTATIONS')]
# cna_blaca_node_pd = oncoprint_blca_node_pd.loc[oncoprint_blca_node_pd['track_type']=='CNA']
blca_cell_line_set = set(oncoprint_blca_pd['cell line'])

In [8]:
# IO compare
def io_compare(wt_io, mut_io):
    """
    wt_io: numpy.ndarray(1, 2**n)
    mut_io: numpy.ndarray(1, 2**n)
    """
    _, num_input_conditions = wt_io.shape
    compare = (wt_io == mut_io)
    ratio = np.sum(compare) / float(num_input_conditions)
    
    return {'compare': compare,
             'ratio': ratio}

In [9]:
# original
Model = jR.grieco()
BooleanRuleFileName = '../'+Model['BooleanRule_filename']
NetworkName = Model['network_name']

NumInputs = Model['num_inputs']
NumInputConditions = Model['num_input_conditions']

InputConditions = Model['input_conditions']

OutputNodes = Model['output_nodes']
InputNodes = Model['input_nodes']

# Set parameters
# Note the node name for Gread is the index (integer), one can encode the nodename by adding prefix and suffix
# If the node name from the input file is not this simple, one need to create a dictionary to record the mapping
Prefix, Suffix = 'n', 'n'

TempI = jR.initialize(BooleanRuleFileName, Prefix, Suffix)
Mapping = TempI['mapping']
InverseMapping = TempI['inverse_mapping']
GRead = TempI['Gread']
ReadNodes = TempI['read_nodes']
if Model['reduction_required']:
    GRead = BDOIp.Get_reduced_network(GRead)

# form expanded network
# GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
# nx.write_gml(GExpanded, NetworkName + '_expanded_network.gml')
try:
    GExpanded = nx.read_gml('../networks/' + NetworkName + '_expanded_network.gml')
except IOError:
    GExpanded = BDOIp.Get_expanded_network(GRead, prefix=Prefix, suffix=Suffix)
    nx.write_gml(GExpanded, '../networks/' + NetworkName + '_expanded_network.gml')

TempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                        constant_nodes=[])
LDOIs = TempGIOW['LDOIs']
GeneLDOIs = TempGIOW['gene_LDOIs']
Conflicts = TempGIOW['conflicts']
GeneConflicts = TempGIOW['gene_conflicts']
IORelation = TempGIOW['io_relation']
GRemained = TempGIOW['G_remained']

In [10]:
# blca_cell_line_set = set(cna_blca_mutation_pd['cell line'])
drugs = ['~AKT', '~BCL2', '~FGFR3', '~MAP3K1_3', '~PLCG', '~PPP2CA', '~RAS', '~SOS', '~ATF2', '~ATM', '~ELK1', '~FOS', '~FRS2', '~GAB1', '~GRB2', '~JUN', '~PKC', '~RAF', '~RSK', '~TAOK']

In [11]:
result = pd.DataFrame(columns=(['cell line', 'mut_io_ratio'] + drugs))
for cell_line in blca_cell_line_set:
    temp_dict = dict()
    cell_line_pd = oncoprint_blca_pd.loc[oncoprint_blca_pd['cell line'] == cell_line, ['track_name', 'event', 'node_name', 'mutation_sign', 'oncoKB']]

#     alteration_set = set()
#     cell_line_pd['alteration'] = 100 * (cell_line_pd['event'] == 'Amplification') - 100 * (cell_line_pd['event'] == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign']
    cell_line_pd['alteration'] = 100 * (cell_line_pd['event'].to_numpy() == 'Amplification') - 100 * (cell_line_pd['event'].to_numpy()  == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign'].to_numpy() 
    cell_line_pd['count'] = 1
    cell_line_pivot_pd = pd.pivot_table(cell_line_pd, index='node_name', values=['alteration', 'count'], aggfunc=np.sum)
#     print(cell_line_pivot_pd.head(15))
    cell_line_pivot_reset_pd = cell_line_pivot_pd.reset_index()
    gof = list(cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] > 0, 'node_name'])
    lof = ['~'+x for x in cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] < 0, 'node_name']]
    # alteration_set = set(gof).union(set(lof))
#     print(alteration_set)

    MutTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=gof+lof)
    mut_io = MutTempGIOW['io_relation']
    mut_io_compare = io_compare(IORelation, mut_io)
    temp_dict['cell line'] = cell_line
    temp_dict['mut_io_ratio'] = mut_io_compare['ratio']
    for drug in drugs:
        
        gof_rc = [x for x in gof if x is not drug[1:]]
        lof_rc = lof + [drug]
        RCTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                                constant_nodes=gof_rc + lof_rc)
        rc_io = RCTempGIOW['io_relation']
        rc_io_compare = io_compare(IORelation, rc_io)
        temp_dict[drug] = rc_io_compare['ratio']

    result = result.append(pd.Series(temp_dict), ignore_index=True)

In [12]:
# result.to_csv('BLCA_CCLE_OneDrug.csv')

In [13]:
from itertools import combinations
drug_two_combinations = list(combinations(drugs,2))
print(drug_two_combinations)

[('~AKT', '~BCL2'), ('~AKT', '~FGFR3'), ('~AKT', '~MAP3K1_3'), ('~AKT', '~PLCG'), ('~AKT', '~PPP2CA'), ('~AKT', '~RAS'), ('~AKT', '~SOS'), ('~AKT', '~ATF2'), ('~AKT', '~ATM'), ('~AKT', '~ELK1'), ('~AKT', '~FOS'), ('~AKT', '~FRS2'), ('~AKT', '~GAB1'), ('~AKT', '~GRB2'), ('~AKT', '~JUN'), ('~AKT', '~PKC'), ('~AKT', '~RAF'), ('~AKT', '~RSK'), ('~AKT', '~TAOK'), ('~BCL2', '~FGFR3'), ('~BCL2', '~MAP3K1_3'), ('~BCL2', '~PLCG'), ('~BCL2', '~PPP2CA'), ('~BCL2', '~RAS'), ('~BCL2', '~SOS'), ('~BCL2', '~ATF2'), ('~BCL2', '~ATM'), ('~BCL2', '~ELK1'), ('~BCL2', '~FOS'), ('~BCL2', '~FRS2'), ('~BCL2', '~GAB1'), ('~BCL2', '~GRB2'), ('~BCL2', '~JUN'), ('~BCL2', '~PKC'), ('~BCL2', '~RAF'), ('~BCL2', '~RSK'), ('~BCL2', '~TAOK'), ('~FGFR3', '~MAP3K1_3'), ('~FGFR3', '~PLCG'), ('~FGFR3', '~PPP2CA'), ('~FGFR3', '~RAS'), ('~FGFR3', '~SOS'), ('~FGFR3', '~ATF2'), ('~FGFR3', '~ATM'), ('~FGFR3', '~ELK1'), ('~FGFR3', '~FOS'), ('~FGFR3', '~FRS2'), ('~FGFR3', '~GAB1'), ('~FGFR3', '~GRB2'), ('~FGFR3', '~JUN'), ('~FGF

In [14]:
result2 = pd.DataFrame(columns=(['cell line', 'mut_io', 'mut_io_ratio'] + [str(x) for x in drug_two_combinations]))
for cell_line in blca_cell_line_set:
    temp_dict = dict()
    cell_line_pd = oncoprint_blca_pd.loc[oncoprint_blca_pd['cell line'] == cell_line, ['track_name', 'event', 'node_name', 'mutation_sign', 'oncoKB']]

#     alteration_set = set()
#     cell_line_pd['alteration'] = 100 * (cell_line_pd['event'] == 'Amplification') - 100 * (cell_line_pd['event'] == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign']
    cell_line_pd['alteration'] = 100 * (cell_line_pd['event'].to_numpy() == 'Amplification') - 100 * (cell_line_pd['event'].to_numpy()  == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign'].to_numpy() 
    cell_line_pd['count'] = 1
    cell_line_pivot_pd = pd.pivot_table(cell_line_pd, index='node_name', values=['alteration', 'count'], aggfunc=np.sum)
#     print(cell_line_pivot_pd.head(15))
    cell_line_pivot_reset_pd = cell_line_pivot_pd.reset_index()
    gof = list(cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] > 0, 'node_name'])
    lof = ['~'+x for x in cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] < 0, 'node_name']]
    # alteration_set = set(gof).union(set(lof))
#     print(alteration_set)

    MutTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=gof+lof)
    mut_io = MutTempGIOW['io_relation']
    mut_io_compare = io_compare(IORelation, mut_io)
    temp_dict['cell line'] = cell_line
    temp_dict['mut_io_ratio'] = mut_io_compare['ratio']
    temp_dict['mut_io'] = mut_io
    for drug_combi in drug_two_combinations:
        drug_targets = [x[1:] for x in drug_combi]
        gof_rc = [x for x in gof if x not in drug_targets]
        lof_rc = lof + list(drug_combi)
        RCTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                                constant_nodes=gof_rc + lof_rc)
        rc_io = RCTempGIOW['io_relation']
        rc_io_compare = io_compare(IORelation, rc_io)
        temp_dict[str(drug_combi)] = rc_io_compare['ratio']

    result2 = result2.append(pd.Series(temp_dict), ignore_index=True)

In [15]:
# result2.to_csv('BLCA_CCLE_TwoDrug.csv')

In [16]:
result2.head(2)

,cell line,mut_io,mut_io_ratio,"('~AKT', '~BCL2')","('~AKT', '~FGFR3')","('~AKT', '~MAP3K1_3')","('~AKT', '~PLCG')","('~AKT', '~PPP2CA')","('~AKT', '~RAS')","('~AKT', '~SOS')",...,"('~JUN', '~PKC')","('~JUN', '~RAF')","('~JUN', '~RSK')","('~JUN', '~TAOK')","('~PKC', '~RAF')","('~PKC', '~RSK')","('~PKC', '~TAOK')","('~RAF', '~RSK')","('~RAF', '~TAOK')","('~RSK', '~TAOK')"
0,T24,"[[[~Apoptosis, ~Growth_Arrest], [~Apoptosis, ~...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,KMBC2,"[[[~Apoptosis, ~Growth_Arrest], [~Apoptosis, ~...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
c2_drugs = ['~EGFR', '~ERK', '~JNK', '~MEK1_2']

In [18]:
result3 = pd.DataFrame(columns=['cell line', 'condition', 'io_ratio'])
for cell_line in blca_cell_line_set:
    temp_dict = dict()
    cell_line_pd = oncoprint_blca_pd.loc[oncoprint_blca_pd['cell line'] == cell_line, ['track_name', 'event', 'node_name', 'mutation_sign', 'oncoKB']]

#     alteration_set = set()
#     cell_line_pd['alteration'] = 100 * (cell_line_pd['event'] == 'Amplification') - 100 * (cell_line_pd['event'] == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign']
    cell_line_pd['alteration'] = 100 * (cell_line_pd['event'].to_numpy() == 'Amplification') - 100 * (cell_line_pd['event'].to_numpy()  == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign'].to_numpy() 
    cell_line_pd['count'] = 1
    cell_line_pivot_pd = pd.pivot_table(cell_line_pd, index='node_name', values=['alteration', 'count'], aggfunc=np.sum)
#     print(cell_line_pivot_pd.head(15))
    cell_line_pivot_reset_pd = cell_line_pivot_pd.reset_index()
    gof = list(cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] > 0, 'node_name'])
    lof = ['~'+x for x in cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] < 0, 'node_name']]
    # alteration_set = set(gof).union(set(lof))
#     print(alteration_set)

    MutTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=gof+lof)
    mut_io = MutTempGIOW['io_relation']
    mut_io_compare = io_compare(IORelation, mut_io)
    temp_dict['cell line'] = cell_line
    temp_dict['condition'] = 'control'
    temp_dict['io_ratio'] = mut_io_compare['ratio']
    result3 = result3.append(pd.Series(temp_dict), ignore_index=True)
    for c2_i in c2_drugs:
        for drug in drugs:
            drug_targets = [x[1:] for x in [c2_i, drug]]
            gof_rc = [x for x in gof if x not in drug_targets]
            lof_rc = lof + [c2_i, drug]
#             gof_rc = [x for x in gof if x is not drug[1:]]
#             lof_rc = lof + [drug]
            RCTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                                    constant_nodes=gof_rc + lof_rc)
            rc_io = RCTempGIOW['io_relation']
            rc_io_compare = io_compare(IORelation, rc_io)
            temp_dict['cell line'] = cell_line
            temp_dict['condition'] = str([c2_i, drug])
            temp_dict['io_ratio'] = rc_io_compare['ratio']
            result3 = result3.append(pd.Series(temp_dict), ignore_index=True)

In [19]:
# result3.to_csv('BLCA_CCLE_C2i_OneDrug.csv')

In [20]:
result4 = pd.DataFrame(columns=['cell line', 'condition', 'io_ratio'])
for cell_line in blca_cell_line_set:
    temp_dict = dict()
    cell_line_pd = oncoprint_blca_pd.loc[oncoprint_blca_pd['cell line'] == cell_line, ['track_name', 'event', 'node_name', 'mutation_sign', 'oncoKB']]

#     alteration_set = set()
#     cell_line_pd['alteration'] = 100 * (cell_line_pd['event'] == 'Amplification') - 100 * (cell_line_pd['event'] == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign']
    cell_line_pd['alteration'] = 100 * (cell_line_pd['event'].to_numpy() == 'Amplification') - 100 * (cell_line_pd['event'].to_numpy()  == 'Deep Deletion') - 10 * (cell_line_pd['event'].str.contains('Truncating mutation')) + 1 * (cell_line_pd['event'].str.contains('Missense Mutation')) * cell_line_pd['mutation_sign'].to_numpy() 
    cell_line_pd['count'] = 1
    cell_line_pivot_pd = pd.pivot_table(cell_line_pd, index='node_name', values=['alteration', 'count'], aggfunc=np.sum)
#     print(cell_line_pivot_pd.head(15))
    cell_line_pivot_reset_pd = cell_line_pivot_pd.reset_index()
    gof = list(cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] > 0, 'node_name'])
    lof = ['~'+x for x in cell_line_pivot_reset_pd.loc[cell_line_pivot_reset_pd['alteration'] < 0, 'node_name']]
    # alteration_set = set(gof).union(set(lof))
#     print(alteration_set)

    MutTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                            constant_nodes=gof+lof)
    mut_io = MutTempGIOW['io_relation']
    mut_io_compare = io_compare(IORelation, mut_io)
    temp_dict['cell line'] = cell_line
    temp_dict['condition'] = 'control'
    temp_dict['io_ratio'] = mut_io_compare['ratio']
    result4 = result4.append(pd.Series(temp_dict), ignore_index=True)
    for c2_i in c2_drugs:
        for drug_combi in drug_two_combinations:
            interventions = [c2_i] + list(drug_combi)
            drug_targets = [x[1:] for x in interventions]
            
            gof_rc = [x for x in gof if x not in drug_targets]
            lof_rc = lof + interventions
            RCTempGIOW = jR.get_input_output_relation(GExpanded, Mapping, InverseMapping, InputConditions, OutputNodes,
                                                    constant_nodes=gof_rc + lof_rc)
            rc_io = RCTempGIOW['io_relation']
            rc_io_compare = io_compare(IORelation, rc_io)
#             temp_dict[str(drug_combi)] = rc_io_compare['ratio']
        
            temp_dict['cell line'] = cell_line
            temp_dict['condition'] = str(interventions)
            temp_dict['io_ratio'] = rc_io_compare['ratio']
            result4 = result4.append(pd.Series(temp_dict), ignore_index=True)

In [21]:
# result4.to_csv('BLCA_CCLE_C2i_TwoDrug.csv')

In [25]:
oncoprint_blca_pd

,Case ID,track_name,event,track_type,gene_symbol,node_name,oncoKB,node,Class,mutation_sign,cell line
14,KU1919_URINARY_TRACT,AKT1,Missense Mutation (putative driver),MUTATIONS,AKT1,AKT,Oncogene,AKT,C1,1,KU1919
165,JMSU1_URINARY_TRACT,ATM,Truncating mutation (putative driver),MUTATIONS,ATM,ATM,Tumor Suppressor,ATM,C0,-1,JMSU1
202,UMUC3_URINARY_TRACT,ATM,Truncating mutation (putative driver),MUTATIONS,ATM,ATM,Tumor Suppressor,ATM,C0,-1,UMUC3
221,HT1376_URINARY_TRACT,CDKN1A,Truncating mutation (putative driver),MUTATIONS,CDKN1A,p21,Tumor Suppressor,p21,C3,-1,HT1376
222,KMBC2_URINARY_TRACT,CDKN1A,Truncating mutation (putative driver),MUTATIONS,CDKN1A,p21,Tumor Suppressor,p21,C3,-1,KMBC2
223,RT112_URINARY_TRACT,CDKN1A,Truncating mutation (putative driver),MUTATIONS,CDKN1A,p21,Tumor Suppressor,p21,C3,-1,RT112
293,SCABER_URINARY_TRACT,CDKN2A,Truncating mutation (putative driver),MUTATIONS,CDKN2A,p14,Tumor Suppressor,p14,C3,-1,SCABER
307,VMCUB1_URINARY_TRACT,CDKN2A,Missense Mutation (putative driver),MUTATIONS,CDKN2A,p14,Tumor Suppressor,p14,C3,-1,VMCUB1
349,RT112_URINARY_TRACT,FGFR3,Amplification,CNA,FGFR3,FGFR3,Oncogene,FGFR3,C1,1,RT112
350,J82_URINARY_TRACT,FGFR3,Missense Mutation (putative driver),MUTATIONS,FGFR3,FGFR3,Oncogene,FGFR3,C1,1,J82


In [28]:
test = pd.pivot_table(oncoprint_blca_pd, index='cell line', columns='track_name', values='mutation_sign', aggfunc=np.sum)

In [30]:
test.to_csv('BLCA_CCLE_alteration.csv')

In [42]:
oncoprint_mrna_melt_pd[oncoprint_mut_melt_pd['Case ID'] == 'KU1919_URINARY_TRACT']

,track_name,track_type,Case ID,event
67792,ATF2,MRNA,KU1919_URINARY_TRACT,NaN
67793,ATM,MRNA,KU1919_URINARY_TRACT,NaN
67794,ELK1,MRNA,KU1919_URINARY_TRACT,NaN
67795,FOS,MRNA,KU1919_URINARY_TRACT,NaN
67796,FRS2,MRNA,KU1919_URINARY_TRACT,NaN
67797,GAB1,MRNA,KU1919_URINARY_TRACT,NaN
67798,GRB2,MRNA,KU1919_URINARY_TRACT,NaN
67799,JUN,MRNA,KU1919_URINARY_TRACT,NaN
67800,PRKCA,MRNA,KU1919_URINARY_TRACT,NaN
67801,PRKCB,MRNA,KU1919_URINARY_TRACT,NaN
